In [1]:
from keras import backend as K
import tensorflow as tf
from tensorflow.python import saved_model
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def

Using TensorFlow backend.


In [2]:
tf.__version__

'2.1.0'

In [3]:
trained_model = 'out7.h5'
backbone ='resnet50'
nb_clacess= 3

export_path = 'retinanet_savedmodel'

In [4]:
tf.compat.v1.disable_eager_execution()

In [5]:
from keras_retinanet import models

import shutil
import os

In [6]:
model = models.convert_model(
    model=models.backbone(backbone_name=backbone).retinanet(num_classes=nb_clacess),
    nms=True,
    class_specific_filter=True,
    anchor_params=None
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


In [7]:
model.load_weights(trained_model)

In [8]:
if os.path.isdir(export_path):
    shutil.rmtree(export_path)

In [9]:
builder = saved_model.builder.SavedModelBuilder(export_path)

In [10]:
signature = predict_signature_def(
    inputs={'images': model.input},
    outputs={
        'output1': model.outputs[0],
        'output2': model.outputs[1],
        'output3': model.outputs[2]
    }
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [11]:
sess = tf.compat.v1.keras.backend.get_session()

In [12]:
builder.add_meta_graph_and_variables(sess=sess,
                                     tags=[saved_model.tag_constants.SERVING],
                                     signature_def_map={'predict': signature})
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: retinanet_savedmodel/saved_model.pb


b'retinanet_savedmodel/saved_model.pb'